<a href="https://colab.research.google.com/github/shawngraham/hist3000/blob/master/archaeological_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started 
We're going to build an archaeological image classifier using Tensorflow, a library of python packages designed to perform machine learning tasks. We are going to train a model to recognize different kinds of Roman pottery. (We are following a modified version of the [Tensorflow for poets](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/#0) tutorial, using slightly older versions of Tensorflow, but a version that I know _works_).

The data we're going to use comes from [Potsherd: Atlas of Roman Pottery](http://potsherd.net/atlas/potsherd). I've already downloaded the dataset, using a bit of python (as described [here](https://bonetrade.github.io/tutorials/tensorflow-for-poets/)). The zip file with those images is available in the course data directory. It's not the full database; rather, I just put together a small set of ten or so different types of pottery, just to keep things manageable. 

Download the zip file, and then upload it to your google drive account. (If you go to https://drive.google.com, you can click the big `+ New`, select `upload file`.)

Don't put it in a subfolder.  

## Make sure we've got a GPU running

Click on `edit` -> `notebook settings` and change `hardware accelerator` to `GPU`. Then click `save`. Machine learning is extremely heavy in the sense that it requires a lot of computational resources. Running the calculations on a GPU rather than a CPU (your normal processor) can speed up the work significantly. And of course, some code explicitly requires a GPU rather than a CPU.

## Connect your gdrive

The snippet below will make your gdrive available to this notebook. Run the block. It will give you a link to go to, which will ask you to authorize access. Say 'yes'. It will then give you a code. Copy that code, then come back to this block and paste it in the entry box. Once you've done that, your gdrive will become available in the `Files` panel at left.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Copy the zip file from your gdrive, and then unzip it

The next code block copies the zipped folder you uploaded to your gdrive into this notebook environment, and then unzips the contents.


In [ ]:
# If you have data already on google drive
%cp "/content/drive/My Drive/data.zip" data.zip

!unzip data.zip

---

# Get Tensorflow Set Up

The version we're going to use is 1.7. It's not the most recent version, but I know it works.


In [21]:
! pip install --upgrade "tensorflow==1.7.*"

     |████████████████████████████████| 48.1MB 63kB/s 
     |████████████████████████████████| 3.1MB 51.7MB/s 
     |████████████████████████████████| 890kB 45.4MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=54e315c0c6d6b18505e30fe214625f0215d0578f8eb0b7669cf1592a9eaff2ec
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


Now we get the codebase that we'll use from the tensorflow for poets repo:

In [3]:
!git clone https://github.com/googlecodelabs/tensorflow-for-poets-2



Cloning into 'tensorflow-for-poets-2'...
remote: Enumerating objects: 426, done.
remote: Total 426 (delta 0), reused 0 (delta 0), pack-reused 426
Receiving objects: 100% (426/426), 33.97 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [ ]:
# and we change our working directory:
%cd tensorflow-for-poets-2


# Retrain the model
Now let's do some training so that the computer learns what Roman archaeology looks like. Because our pottery training data - though copious - is still not enough (fewer than 10000 images), we have to modify our command, concerning validation batch size. Otherwise we’ll get an error message. Note also that we’re only training for 500 steps; more steps will generally get better results (but diminishing returns also apply).

In [ ]:
!python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/mobilenet_0.50_224 \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture mobilenet_0.50_224 \
  --validation_batch_size=-1 \
  --image_dir=../data/ware # this is the location of our data

# Test!

This is the fun part. In the data I provided, there is a folder called `unknown`. It has some photos of pottery in there. Let's get our model to identify the pottery!

In [25]:
# let's grab a pot that we know is black burnished ware:
!curl "http://www.pottedhistory.co.uk/image/Roman%20Black%20Burnished%20Ware(800x532).jpg" > bbtest.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114k  100  114k    0     0   492k      0 --:--:-- --:--:-- --:--:--  490k


In [ ]:
!python -m scripts.label_image \
    --graph=tf_files/retrained_graph.pb  \
    --image=bbtest.jpg

## there will be some warnings. But at the end, it will tell you which ware it thinks the photo is of.
## (If you want to know what kind of pottery the code represents, just add the code to the end of
## this url in a new window: http://potsherd.net/atlas/Ware/) eg
## http://potsherd.net/atlas/Ware/BB1 will bring up Black Burnished Ware

In [ ]:
## If that image was _misclassified_, can you think why that might be?
## Try this one:
!curl "http://www.futuremuseum.co.uk/imageGen.ashx?image=%2fmedia%2f3901%2f9083.jpg" > bbtest2.jpg

In [ ]:
!python -m scripts.label_image \
    --graph=tf_files/retrained_graph.pb  \
    --image=bbtest2.jpg

## Success?

We've certainly built an image classifier, but there are some problems. There are both technical issues, and ethical issues (on the ethical issues, see [this article about using this kind of tech in the context of human remains](https://intarch.ac.uk/journal/issue52/5/toc.html). Tech-wise, the biggest issue is that we're training on a dataset that is just _too small_. For this to _really_ work well enough to the point where we might actually want to use it, we'd need thousands of images. There are shortcuts we could use (image augmentation, where we rotate and otherwise make slightly different versions of our source images, for instance). 

Archaeologists are using this basic approach (with much sophistication and nuance) for a wide variety of tasks. [In this paper, they're using it to identify archaeological sites from satellite data.](https://www.pnas.org/content/early/2020/07/17/2005583117).

## Going Further

You could try finding more training data; you could augment the images you do have [with this code](https://colab.research.google.com/drive/1Sxo8mMdmvdzHkNMAcCTxHomHhaCdkIyf?usp=sharing). 